In [2]:
import os
import numpy as np
import cv2
import pandas as pd

path = r"V:\grid_layout"
file = r"combined.csv"

combined_df = pd.read_csv(os.path.join(path, file))
# lines_df = pd.read_csv(os.path.join(path, "lines_data_text_files", "lines_30.txt"), header = None, sep = ",")
combined_df.head()

,Unnamed: 0,text,bbox,shape,image,tag,center
0,0,-212,"[78, 51, 124, 70]",1137 1739,30.png,O,[101. 60.5]
1,1,753-,"[130, 48, 171, 67]",1137 1739,30.png,O,[150.5 57.5]
2,2,4455,"[171, 48, 217, 65]",1137 1739,30.png,O,[194. 56.5]
3,3,6242,"[874, 100, 945, 127]",1137 1739,30.png,O,[909.5 113.5]
4,4,INVOICE,"[657, 105, 747, 133]",1137 1739,30.png,B-QUESTION,[702. 119.]


In [3]:
img = cv2.imread(os.path.join(path, 'raw_data', '30.png'))
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow("img", 480,640)
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
img.shape

(1739, 1137, 3)

# get the max words in one line(column) --> calculate a range and get words in that range
# same for max words in a columns

In [5]:
# with open(os.path.join(path, "lines_data_text_files", "lines_30.txt"), 'r') as f:
#     content = f.readlines()



# Assuming that grid size would 64x64 

In [108]:
grid = np.zeros((64,64), np.uint)

In [7]:
df = combined_df.copy()

In [8]:
df['bbox'] = df['bbox'].apply(eval)

In [9]:
df['bbox'][0]

[78, 51, 124, 70]

In [10]:
df['width'] = df['bbox'].apply(lambda x: x[2] - x[0])
df['height'] = df['bbox'].apply(lambda y: y[3] - y[1])
df['image_height'] = df['shape'].apply(lambda z: z.split(" ")[1])
df['image_width'] = df['shape'].apply(lambda z: z.split(" ")[0])

In [11]:
df['shape'][0].split(" ")

['1137', '1739']

In [12]:
df.head()

,Unnamed: 0,text,bbox,shape,image,tag,center,width,height,image_height,image_width
0,0,-212,"[78, 51, 124, 70]",1137 1739,30.png,O,[101. 60.5],46,19,1739,1137
1,1,753-,"[130, 48, 171, 67]",1137 1739,30.png,O,[150.5 57.5],41,19,1739,1137
2,2,4455,"[171, 48, 217, 65]",1137 1739,30.png,O,[194. 56.5],46,17,1739,1137
3,3,6242,"[874, 100, 945, 127]",1137 1739,30.png,O,[909.5 113.5],71,27,1739,1137
4,4,INVOICE,"[657, 105, 747, 133]",1137 1739,30.png,B-QUESTION,[702. 119.],90,28,1739,1137


In [13]:
def cal_row(n_rows, x_left, text_width, image_width):
    row_i = n_rows*(x_left + (text_width/2))/image_width
    return row_i

def cal_col(n_cols, y_top, text_height, image_height):
    col_i = n_cols*(y_top + (text_height/2))/image_height
    return col_i

In [14]:
rows, cols = [], []
for i in df.index:
    rows.append(cal_row(64, df['bbox'][i][0], df['width'][i], int(df['image_width'][i])))
    cols.append(cal_col(64, df['bbox'][i][2], df['height'][i], int(df['image_height'][i])))
#     break
# print(rows, cols)

In [15]:
df['row'] = rows
df['col'] = cols

In [18]:
import math
df['int_row'] = df['row'].apply(lambda x: int(x))
df['int_col'] = df['col'].apply(lambda x: int(x))
df['floor_row'] = df['row'].apply(lambda x: math.ceil(x))
df['floor_col'] = df['col'].apply(lambda x: math.ceil(x))

In [19]:
df.head()

,Unnamed: 0,text,bbox,shape,image,tag,center,width,height,image_height,image_width,row,col,int_row,int_col,floor_row,floor_col
0,0,-212,"[78, 51, 124, 70]",1137 1739,30.png,O,[101. 60.5],46,19,1739,1137,5.685136,4.913168,5,4,6,5
1,1,753-,"[130, 48, 171, 67]",1137 1739,30.png,O,[150.5 57.5],41,19,1739,1137,8.471416,6.642898,8,6,9,7
2,2,4455,"[171, 48, 217, 65]",1137 1739,30.png,O,[194. 56.5],46,17,1739,1137,10.919965,8.299022,10,8,11,9
3,3,6242,"[874, 100, 945, 127]",1137 1739,30.png,O,[909.5 113.5],71,27,1739,1137,51.194371,35.275446,51,35,52,36
4,4,INVOICE,"[657, 105, 747, 133]",1137 1739,30.png,B-QUESTION,[702. 119.],90,28,1739,1137,39.514512,28.006901,39,28,40,29


In [20]:
df_ = df.copy()
df_ = df_[df['image']=='30.png']

In [21]:
df_.head()

,Unnamed: 0,text,bbox,shape,image,tag,center,width,height,image_height,image_width,row,col,int_row,int_col,floor_row,floor_col
0,0,-212,"[78, 51, 124, 70]",1137 1739,30.png,O,[101. 60.5],46,19,1739,1137,5.685136,4.913168,5,4,6,5
1,1,753-,"[130, 48, 171, 67]",1137 1739,30.png,O,[150.5 57.5],41,19,1739,1137,8.471416,6.642898,8,6,9,7
2,2,4455,"[171, 48, 217, 65]",1137 1739,30.png,O,[194. 56.5],46,17,1739,1137,10.919965,8.299022,10,8,11,9
3,3,6242,"[874, 100, 945, 127]",1137 1739,30.png,O,[909.5 113.5],71,27,1739,1137,51.194371,35.275446,51,35,52,36
4,4,INVOICE,"[657, 105, 747, 133]",1137 1739,30.png,B-QUESTION,[702. 119.],90,28,1739,1137,39.514512,28.006901,39,28,40,29


In [22]:
len(df_['int_row'].unique()), len(df_['int_col'].unique())

(38, 30)

In [23]:
len(df_['floor_row'].unique()), len(df_['floor_col'].unique())

(38, 30)

In [24]:
len(df_['row']), len(df_['col'])

(89, 89)

In [25]:
max(df_['int_row']), max(df_['int_col'])

(57, 40)

In [26]:
min(df_['int_row']), min(df_['int_col'])

(5, 4)

In [27]:
max(df_['floor_row']), max(df_['floor_col'])

(58, 41)

In [28]:
min(df_['floor_row']), min(df_['floor_col'])

(6, 5)

# TODO --> manage conflicting rows and Columns 

In [74]:
len(df_['floor_col']), len(df_['floor_row'])

(89, 89)

In [69]:
z = zip(rows, cols)

In [81]:
df_['pos_r'] = df_['floor_row'].apply(lambda x: str(x))

In [82]:
df_['pos_c'] = df_['floor_col'].apply(lambda x: str(x))

In [90]:
df_['pos'] = df_['pos_r']+","+df_['pos_c']

In [91]:
df_['pos'] = df_['pos'].apply(eval)
df_['pos'][0]

(6, 5)

In [93]:
df_['pos'].unique

<bound method Series.unique of 0       (6, 5)
1       (9, 7)
2      (11, 9)
3     (52, 36)
4     (40, 29)
        ...   
84    (55, 37)
85    (57, 38)
86    (14, 11)
87    (54, 37)
88    (57, 39)
Name: pos, Length: 89, dtype: object>

In [109]:
for i in df_.index:
    grid[df_['pos'].iloc[i]] = df_['Unnamed: 0'].iloc[i]

In [110]:
for i in grid:
    print(i)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0

In [111]:
df_.head()

,Unnamed: 0,text,bbox,shape,image,tag,center,width,height,image_height,image_width,row,col,int_row,int_col,floor_row,floor_col,pos,pos_r,pos_c
0,0,-212,"[78, 51, 124, 70]",1137 1739,30.png,O,[101. 60.5],46,19,1739,1137,5.685136,4.913168,5,4,6,5,"(6, 5)",6,5
1,1,753-,"[130, 48, 171, 67]",1137 1739,30.png,O,[150.5 57.5],41,19,1739,1137,8.471416,6.642898,8,6,9,7,"(9, 7)",9,7
2,2,4455,"[171, 48, 217, 65]",1137 1739,30.png,O,[194. 56.5],46,17,1739,1137,10.919965,8.299022,10,8,11,9,"(11, 9)",11,9
3,3,6242,"[874, 100, 945, 127]",1137 1739,30.png,O,[909.5 113.5],71,27,1739,1137,51.194371,35.275446,51,35,52,36,"(52, 36)",52,36
4,4,INVOICE,"[657, 105, 747, 133]",1137 1739,30.png,B-QUESTION,[702. 119.],90,28,1739,1137,39.514512,28.006901,39,28,40,29,"(40, 29)",40,29


In [115]:
grid[52,36]

3